# Emotion Recognition using CNN

In [1]:
import tensorflow as tf
import os
import cv2
from tensorflow.keras import layers

In [2]:
# consts

# Please provide you own path
path = "../../Datasets/Emotion Recognition/"

train_folder = path + "train/"
valid_folder = path + "validation/"

emotion_labels = ['Happy', 'Surprise', 'Sad', 'Neutral', 'Angry', 'Disgust', 'Fear']


In [3]:
# Defining emotions
class Emotions:
    def __init__(self):
        self.emotion_labels = ['Happy', 'Surprise', 'Sad', 'Neutral', 'Angry', 'Disgust', 'Fear']

    def get_emotion_label(self, index):
        return self.emotion_labels[index]

In [4]:

# Function to load images from a folder
def load_images_from_folder(folder):
    images = []
    labels = []
    for emotion_idx, emotion in enumerate(emotion_labels):
        folder_path = os.path.join(folder, emotion)
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is not None:
                images.append(image)
                labels.append(emotion_idx)
    return images, labels

In [ ]:
# Define the CNN model
model = tf.keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    
    layers.Dense(7, activation='softmax')  # 7 output classes for emotions
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
model.fit(x_train, y_train, validation_data=(x_valid, y_valid), epochs=10, batch_size=32)